## Capstone Project "What's Happening in my Neighborhood"
by Lori Butler  

**Data Questions**
1. Which neighborhoods have had the most, and the least, growth during the past three years based on the count of building permits by type, and by cost of construction.
2. Do any neighborhoods have recent increases in building permit applications which may signal growth in the near future?
3. In instances where zoning changes are sought, might those be a leading indicator of a subsequent increase in building permit applications? If so, what is the average time lag?

**Data Sources**
- All data is through Friday 6/5/2020
- CSV
    - df_bldg_apps = Building Permit Applications, rolling 3 years
    - df_bldg_issued = Building Permits Issues, rolling 3 years
    - df_planning = Planning (Zoning) Department Applications (all pending), and issued (rolling 2 months after issuance)
- GEOJSON
    - df_na_bound = Neighborhood Boundaries GeoJSON (polygon) folders

**Directory Structure:**
- data
    - cleaned = clean files, ready to be used for EDA  
    - interim = temporary files used during cleaning
    - raw = unedited csv and Excel files used as teh source of analysis

- notebooks
    - 1_dataprep = standard cleanup of columns, data types, and additional data manipulation specific to this project (nulls, lat/lon, rolling week/month, etc.)
    - 2_eda = EDA to create reports, and generate clean files to export to visualization tool.
- notes_and_docs = documents created for reference, metadata, etc
- reports = final reports for presentations

*Notebook and data structure inspired by **Practicle Business Python** article by Chris Moffitt https://pbpython.com/notebook-process.html*

## Data Prep 01 Notebook: Column and Data Type Cleanup

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd   # Prerequisite: Anotes_and_docsctivate geospatial environment via Conda Prompt
import matplotlib.pyplot as plt  
import folium                   
from folium.plugins import MarkerCluster

# May not need this until I do EDA, but importing now as a reminder.
from shapely.geometry import Point  

# To use RegEx to pull ot lat/long from building permit applications/issued
import re

## Read in raw files

In [2]:
# Building Dept. Permit Applications

df_bldg_apps = pd.read_csv('../data/raw/Building_Permit_Applications_2020_06_05.csv')
df_bldg_apps.head(2)

Permit #     Permit Type Description   Permit Subtype Description  \
0  T2020016213  Building Residential - New      Single Family Residence   
1  T2019073204      Building Moving Permit  Moving Permit - Residential   

        Parcel Date Entered  Date Issued  Construction Cost  \
0  10216006100   03/11/2020          NaN                NaN   
1  04600002700   12/02/2019          NaN             2500.0   

             Address          City State    ZIP  \
0     748  DARDEN PL     NASHVILLE    TN  37205   
1  4836  BULL RUN RD  ASHLAND CITY    TN  37015   

                        Subdivision / Lot                       Contact  \
0         LOT 168 SEC 9 PT 2 HILLWOOD EST  Kingdom Builders of Tennesse   
1  N OF BULL RUN RD W OF OLD HICKORY BLVD            CLAYTON HOMES #054   

  Permit Type Permit Subtype  IVR Tracking #  \
0        CARN      CAA01R301         3781725   
1        CAMV      CAZ09A001         3736813   

                                             Purpose  Council District  \
0  New Single family dwelling.  REJECTED: APPLICA...              23.0   
1  Move existing mobile home from property out of...               1.0   

                                     Mapped Location  
0  748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...  
1  4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36....

In [3]:
# Confirming shape. OK = 3106 rows (# from data.Nashville.gov website)

df_bldg_apps.shape

(3106, 19)

In [4]:
# Building Dept. Permits Issued
# low_memory = False was added to remove a low-memory warning. Doing this prevents the
# system from trying to assign dtypes until after the full file has been read
# Resource: https://tinyurl.com/stackoverflow-low-memory

df_bldg_issued = pd.read_csv('../data/raw/Building_Permits_Issued_2020_06_05.csv'
                             , low_memory=False)
df_bldg_issued.head(2)

Permit #       Permit Type Description Permit Subtype Description  \
0  2019070460    Building Residential - New    Single Family Residence   
1  2020016259  Building Residential - Rehab    Single Family Residence   

           Parcel Date Entered Date Issued  Construction Cost  \
0  058100C04900CO   11/18/2019  12/09/2019           270585.0   
1  160150A07000CO   03/12/2020  03/12/2020            12000.0   

                     Address       City State    ZIP  \
0     1037  LAWSONS RIDGE DR  NASHVILLE    TN  37218   
1  210  HEARTHSTONE MANOR LN  BRENTWOOD    TN  37027   

                               Subdivision / Lot                Contact  \
0                   LOT 49 CARRINGTON PLACE PH 5  CELEBRATION HOMES LLC   
1  UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4  ACCESS & MOBILITY INC   

  Permit Type Permit Subtype  IVR Tracking #  \
0        CARN      CAA01R301         3733056   
1        CARR      CAA01R301         3781961   

                                             Purpose  Council District  \
0  To construct a single family residence of 2402...               1.0   
1  to install a new elevator/platform lift from g...               4.0   

   Census Tract                                    Mapped Location  
0    37010105.0         1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218  
1    37018803.0  210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...

In [5]:
# Confirming shape. OK = 33909 rows (# from data.Nashville.gov website)

df_bldg_issued.shape

(33909, 20)

In [6]:
# Planning/Zoning Applications & Issued
# Initially read in file through 6/5/2020, but that had wrong dates.

df_planning = pd.read_csv('../data/raw/Planning_Department_Development_Applications_2020_06_05.csv')
df_planning.head(2)

Date Submitted     Application Type Description      MPC Case # Ordinance #  \
0     04/01/2019         Subdivision (Final Plat)   2019S-086-001         NaN   
1     11/27/2019  Specific Plan (Final Site Plan)  2016SP-076-008         NaN   

    Status MPC Meeting Date MPC Action  \
0  PENDING       06/11/2020        NaN   
1  PENDING       01/16/2020        NaN   

                                        Project Name              Location  \
0  FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...  227 MARCIA AVE 37209   
1                                 RED OAKS TOWNHOMES        0 DEW ST 37206   

          Reviewer  ... Applicant Address 2 Applicant City Applicant State  \
0  Joren Dunnavant  ...                 NaN      Nashville              TN   
1    Abbie Rickoff  ...                 NaN      Nashville              TN   

  Applicant ZIP Council 3rd Reading Date Council 3rd Reading Action  \
0         37203                      NaN                        NaN   
1         37204                      NaN                        NaN   

            Council District   Latitude  Longitude  \
0  20 (Mary Carolyn Roberts)  36.143923 -86.868254   
1         06 (Brett Withers)  36.165962 -86.753490   

                            Mapped Location  
0  (36.143922831000054, -86.86825400699996)  
1  (36.165961579000054, -86.75348957099999)  

[2 rows x 27 columns]

In [7]:
# Confirming shape. OK = 521 rows (# from data.Nashville.gov website)

df_planning.shape

(521, 27)

In [8]:
# Neighborhood Assoc boundaries GIS file, using geopandas

df_na_bound = gpd.read_file('../data/raw/Neighborhood Association Boundaries (GIS)_2020_06_03.geojson')
print(df_na_bound.crs)
df_na_bound.head(2)

epsg:4326


name                                           geometry
0  Historic Buena Vista  MULTIPOLYGON (((-86.79511 36.17576, -86.79403 ...
1        Charlotte Park  MULTIPOLYGON (((-86.87460 36.15758, -86.87317 ...

## Column Name Cleanup - Bldg Permit Applications

In [9]:
df_bldg_apps.columns

Index(['Permit #', 'Permit Type Description', 'Permit Subtype Description',
       'Parcel', 'Date Entered', 'Date Issued', 'Construction Cost', 'Address',
       'City', 'State', 'ZIP', 'Subdivision / Lot', 'Contact', 'Permit Type',
       'Permit Subtype', 'IVR Tracking #', 'Purpose', 'Council District',
       'Mapped Location'],
      dtype='object')

In [10]:
df_bldg_apps.columns = (df_bldg_apps.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_bldg_apps.head(2)

permit_number           permit_type_descr         permit_subtype_descr  \
0   T2020016213  Building Residential - New      Single Family Residence   
1   T2019073204      Building Moving Permit  Moving Permit - Residential   

        parcel date_entered  date_issued  construction_cost  \
0  10216006100   03/11/2020          NaN                NaN   
1  04600002700   12/02/2019          NaN             2500.0   

             address          city state    zip  \
0     748  DARDEN PL     NASHVILLE    TN  37205   
1  4836  BULL RUN RD  ASHLAND CITY    TN  37015   

                        subdivision___lot                       contact  \
0         LOT 168 SEC 9 PT 2 HILLWOOD EST  Kingdom Builders of Tennesse   
1  N OF BULL RUN RD W OF OLD HICKORY BLVD            CLAYTON HOMES #054   

  permit_type permit_subtype  ivr_tracking_number  \
0        CARN      CAA01R301              3781725   
1        CAMV      CAZ09A001              3736813   

                                             purpose  council_district  \
0  New Single family dwelling.  REJECTED: APPLICA...              23.0   
1  Move existing mobile home from property out of...               1.0   

                                     mapped_location  
0  748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...  
1  4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36....

In [11]:
# To correct issue with too many underscores in subdivision_lot column name

df_bldg_apps = df_bldg_apps.rename(columns = {'subdivision___lot': 'subdivision_lot'})
df_bldg_apps.columns

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision_lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'mapped_location'],
      dtype='object')

## Column Name Cleanup -  Bldg Permits Issued

In [12]:
df_bldg_issued.columns

Index(['Permit #', 'Permit Type Description', 'Permit Subtype Description',
       'Parcel', 'Date Entered', 'Date Issued', 'Construction Cost', 'Address',
       'City', 'State', 'ZIP', 'Subdivision / Lot', 'Contact', 'Permit Type',
       'Permit Subtype', 'IVR Tracking #', 'Purpose', 'Council District',
       'Census Tract', 'Mapped Location'],
      dtype='object')

In [13]:
df_bldg_issued.columns = (df_bldg_issued.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_bldg_issued.head(2)

permit_number             permit_type_descr     permit_subtype_descr  \
0    2019070460    Building Residential - New  Single Family Residence   
1    2020016259  Building Residential - Rehab  Single Family Residence   

           parcel date_entered date_issued  construction_cost  \
0  058100C04900CO   11/18/2019  12/09/2019           270585.0   
1  160150A07000CO   03/12/2020  03/12/2020            12000.0   

                     address       city state    zip  \
0     1037  LAWSONS RIDGE DR  NASHVILLE    TN  37218   
1  210  HEARTHSTONE MANOR LN  BRENTWOOD    TN  37027   

                               subdivision___lot                contact  \
0                   LOT 49 CARRINGTON PLACE PH 5  CELEBRATION HOMES LLC   
1  UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4  ACCESS & MOBILITY INC   

  permit_type permit_subtype  ivr_tracking_number  \
0        CARN      CAA01R301              3733056   
1        CARR      CAA01R301              3781961   

                                             purpose  council_district  \
0  To construct a single family residence of 2402...               1.0   
1  to install a new elevator/platform lift from g...               4.0   

   census_tract                                    mapped_location  
0    37010105.0         1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218  
1    37018803.0  210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...

In [14]:
# To correct issue with too many underscores in subdivision_lot column name

df_bldg_issued = df_bldg_issued.rename(columns = {'subdivision___lot': 'subdivision_lot'})
df_bldg_issued.columns

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'date_issued', 'construction_cost', 'address', 'city',
       'state', 'zip', 'subdivision_lot', 'contact', 'permit_type',
       'permit_subtype', 'ivr_tracking_number', 'purpose', 'council_district',
       'census_tract', 'mapped_location'],
      dtype='object')

## Column Name Cleanup -  Planning Dept

In [15]:
df_planning.columns

Index(['Date Submitted', 'Application Type Description', 'MPC Case #',
       'Ordinance #', 'Status', 'MPC Meeting Date', 'MPC Action',
       'Project Name', 'Location', 'Reviewer', 'Reviewer Email',
       'Case Description', 'Applicant', 'Applicant Representative',
       'Applicant Email', 'Applicant Phone', 'Applicant Address 1',
       'Applicant Address 2', 'Applicant City', 'Applicant State',
       'Applicant ZIP', 'Council 3rd Reading Date',
       'Council 3rd Reading Action', 'Council District', 'Latitude',
       'Longitude', 'Mapped Location'],
      dtype='object')

In [16]:
df_planning.columns = (df_planning.columns
                        .str.replace(" ", "_")
                        .str.replace("/", "_")
                        .str.replace("Description", "descr")
                        .str.replace("#", "number")
                        .str.lower())
df_planning.head(2)

date_submitted           application_type_descr mpc_case_number  \
0     04/01/2019         Subdivision (Final Plat)   2019S-086-001   
1     11/27/2019  Specific Plan (Final Site Plan)  2016SP-076-008   

  ordinance_number   status mpc_meeting_date mpc_action  \
0              NaN  PENDING       06/11/2020        NaN   
1              NaN  PENDING       01/16/2020        NaN   

                                        project_name              location  \
0  FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...  227 MARCIA AVE 37209   
1                                 RED OAKS TOWNHOMES        0 DEW ST 37206   

          reviewer  ... applicant_address_2 applicant_city applicant_state  \
0  Joren Dunnavant  ...                 NaN      Nashville              TN   
1    Abbie Rickoff  ...                 NaN      Nashville              TN   

  applicant_zip council_3rd_reading_date council_3rd_reading_action  \
0         37203                      NaN                        NaN   
1         37204                      NaN                        NaN   

            council_district   latitude  longitude  \
0  20 (Mary Carolyn Roberts)  36.143923 -86.868254   
1         06 (Brett Withers)  36.165962 -86.753490   

                            mapped_location  
0  (36.143922831000054, -86.86825400699996)  
1  (36.165961579000054, -86.75348957099999)  

[2 rows x 27 columns]

### Data type cleanup

#### Bldg Permit Applications - dtype cleanup

In [17]:
df_bldg_apps.head(2)

permit_number           permit_type_descr         permit_subtype_descr  \
0   T2020016213  Building Residential - New      Single Family Residence   
1   T2019073204      Building Moving Permit  Moving Permit - Residential   

        parcel date_entered  date_issued  construction_cost  \
0  10216006100   03/11/2020          NaN                NaN   
1  04600002700   12/02/2019          NaN             2500.0   

             address          city state    zip  \
0     748  DARDEN PL     NASHVILLE    TN  37205   
1  4836  BULL RUN RD  ASHLAND CITY    TN  37015   

                          subdivision_lot                       contact  \
0         LOT 168 SEC 9 PT 2 HILLWOOD EST  Kingdom Builders of Tennesse   
1  N OF BULL RUN RD W OF OLD HICKORY BLVD            CLAYTON HOMES #054   

  permit_type permit_subtype  ivr_tracking_number  \
0        CARN      CAA01R301              3781725   
1        CAMV      CAZ09A001              3736813   

                                             purpose  council_district  \
0  New Single family dwelling.  REJECTED: APPLICA...              23.0   
1  Move existing mobile home from property out of...               1.0   

                                     mapped_location  
0  748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...  
1  4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36....

In [18]:
# Confirming dtypes
# NEED TO CHANGE: date_entered should be datetime fields (date only).

df_bldg_apps.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         3106 non-null   object 
 1   permit_type_descr     3106 non-null   object 
 2   permit_subtype_descr  3106 non-null   object 
 3   parcel                3106 non-null   object 
 4   date_entered          3106 non-null   object 
 5   date_issued           0 non-null      float64
 6   construction_cost     1651 non-null   float64
 7   address               3106 non-null   object 
 8   city                  3106 non-null   object 
 9   state                 3106 non-null   object 
 10  zip                   3106 non-null   int64  
 11  subdivision_lot       3105 non-null   object 
 12  contact               3105 non-null   object 
 13  permit_type           3106 non-null   object 
 14  permit_subtype        3106 non-null   object 
 15  ivr_tracking_number  

In [19]:
# All values in date_issued are null. Will remove this from df

df_bldg_apps.isnull().sum()

permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
date_issued             3106
construction_cost       1455
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            1
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                   22
council_district           7
mapped_location            0
dtype: int64

In [20]:
# To drop date_issued from building applications df (all values are null)

df_bldg_apps = df_bldg_apps.drop(columns = ['date_issued'])

In [21]:
# To confirm column dropped successfully. DONE!

df_bldg_apps.columns  

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'construction_cost', 'address', 'city', 'state', 'zip',
       'subdivision_lot', 'contact', 'permit_type', 'permit_subtype',
       'ivr_tracking_number', 'purpose', 'council_district',
       'mapped_location'],
      dtype='object')

In [22]:
# Convert date_entered to datetimee.

df_bldg_apps.date_entered = pd.to_datetime(df_bldg_apps.date_entered)
df_bldg_apps.date_entered.head(2)

0   2020-03-11
1   2019-12-02
Name: date_entered, dtype: datetime64[ns]

In [23]:
# Double-checking min/max dates in this df
# .dt.date causes only the date to show

print(df_bldg_apps.date_entered.min())
print(df_bldg_apps.date_entered.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


#### Bldg Permits Issued - dtype cleanup

In [24]:
df_bldg_issued.head(2)

permit_number             permit_type_descr     permit_subtype_descr  \
0    2019070460    Building Residential - New  Single Family Residence   
1    2020016259  Building Residential - Rehab  Single Family Residence   

           parcel date_entered date_issued  construction_cost  \
0  058100C04900CO   11/18/2019  12/09/2019           270585.0   
1  160150A07000CO   03/12/2020  03/12/2020            12000.0   

                     address       city state    zip  \
0     1037  LAWSONS RIDGE DR  NASHVILLE    TN  37218   
1  210  HEARTHSTONE MANOR LN  BRENTWOOD    TN  37027   

                                 subdivision_lot                contact  \
0                   LOT 49 CARRINGTON PLACE PH 5  CELEBRATION HOMES LLC   
1  UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4  ACCESS & MOBILITY INC   

  permit_type permit_subtype  ivr_tracking_number  \
0        CARN      CAA01R301              3733056   
1        CARR      CAA01R301              3781961   

                                             purpose  council_district  \
0  To construct a single family residence of 2402...               1.0   
1  to install a new elevator/platform lift from g...               4.0   

   census_tract                                    mapped_location  
0    37010105.0         1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218  
1    37018803.0  210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...

In [25]:
# Confirming dtypes
# NEED TO CHANGE: date_entered and date_issued should be datetime fields (date only).

df_bldg_issued.info()  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33909 entries, 0 to 33908
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   permit_number         33909 non-null  object 
 1   permit_type_descr     33909 non-null  object 
 2   permit_subtype_descr  33909 non-null  object 
 3   parcel                33909 non-null  object 
 4   date_entered          33909 non-null  object 
 5   date_issued           33909 non-null  object 
 6   construction_cost     33899 non-null  float64
 7   address               33909 non-null  object 
 8   city                  33909 non-null  object 
 9   state                 33909 non-null  object 
 10  zip                   33909 non-null  int64  
 11  subdivision_lot       33909 non-null  object 
 12  contact               33908 non-null  object 
 13  permit_type           33909 non-null  object 
 14  permit_subtype        33909 non-null  object 
 15  ivr_tracking_number

In [26]:
# Convert date_entered to datetime

df_bldg_issued.date_entered = pd.to_datetime(df_bldg_issued.date_entered)
df_bldg_issued.date_entered.head(2)

0   2019-11-18
1   2020-03-12
Name: date_entered, dtype: datetime64[ns]

In [27]:
# Convert date_issued to datetime, keeping DATE only, not time.
df_bldg_issued.date_issued = pd.to_datetime(df_bldg_issued.date_issued)
df_bldg_issued.date_issued.head(2)

0   2019-12-09
1   2020-03-12
Name: date_issued, dtype: datetime64[ns]

In [28]:
# Double-checking min/max dates in this df

print(df_bldg_issued.date_issued.min())
print(df_bldg_issued.date_issued.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


#### Planning Dept Applications & Issued - dtype cleanup

In [29]:
df_planning.head(2)

date_submitted           application_type_descr mpc_case_number  \
0     04/01/2019         Subdivision (Final Plat)   2019S-086-001   
1     11/27/2019  Specific Plan (Final Site Plan)  2016SP-076-008   

  ordinance_number   status mpc_meeting_date mpc_action  \
0              NaN  PENDING       06/11/2020        NaN   
1              NaN  PENDING       01/16/2020        NaN   

                                        project_name              location  \
0  FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...  227 MARCIA AVE 37209   
1                                 RED OAKS TOWNHOMES        0 DEW ST 37206   

          reviewer  ... applicant_address_2 applicant_city applicant_state  \
0  Joren Dunnavant  ...                 NaN      Nashville              TN   
1    Abbie Rickoff  ...                 NaN      Nashville              TN   

  applicant_zip council_3rd_reading_date council_3rd_reading_action  \
0         37203                      NaN                        NaN   
1         37204                      NaN                        NaN   

            council_district   latitude  longitude  \
0  20 (Mary Carolyn Roberts)  36.143923 -86.868254   
1         06 (Brett Withers)  36.165962 -86.753490   

                            mapped_location  
0  (36.143922831000054, -86.86825400699996)  
1  (36.165961579000054, -86.75348957099999)  

[2 rows x 27 columns]

In [30]:
df_planning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   date_submitted              521 non-null    object 
 1   application_type_descr      521 non-null    object 
 2   mpc_case_number             520 non-null    object 
 3   ordinance_number            157 non-null    object 
 4   status                      521 non-null    object 
 5   mpc_meeting_date            521 non-null    object 
 6   mpc_action                  235 non-null    object 
 7   project_name                417 non-null    object 
 8   location                    498 non-null    object 
 9   reviewer                    521 non-null    object 
 10  reviewer_email              465 non-null    object 
 11  case_descr                  503 non-null    object 
 12  applicant                   501 non-null    object 
 13  applicant_representative    497 non

In [31]:
df_planning.date_submitted = pd.to_datetime(df_planning.date_submitted)
df_planning.date_submitted.head(2)

0   2019-04-01
1   2019-11-27
Name: date_submitted, dtype: datetime64[ns]

In [32]:
df_planning.mpc_meeting_date = pd.to_datetime(df_planning.mpc_meeting_date)
df_planning.mpc_meeting_date.head(2)

0   2020-06-11
1   2020-01-16
Name: mpc_meeting_date, dtype: datetime64[ns]

In [33]:
# Double-checking min/max dates in this df

print(df_planning.date_submitted.min())   #2017-02-28
print(df_planning.date_submitted.max())   #2020-06-04
print(df_planning.mpc_meeting_date.min())  #2017-04-13
print(df_planning.mpc_meeting_date.max())  #2020-07-23 - Future date is correct

2017-02-28 00:00:00
2020-06-04 00:00:00
2017-04-13 00:00:00
2020-07-23 00:00:00


## Fix mapped_location, pull out lat/lon in Bldg Permit Apps & Issued

In [34]:
# Building Permit Applications

df_bldg_apps.mapped_location.unique()

array(['748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944, -86.879062)',
       '4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36.242681, -86.929594)',
       '4119 MURFREESBORO PIKE\nANTIOCH, TN 37013\n(36.032211, -86.594799)',
       ...,
       '6680 CHARLOTTE PIKE B-5\nNASHVILLE, TN 37209\n(36.136609, -86.883701)',
       '3805 CHARLOTTE AVE\nNASHVILLE, TN 37209\n(36.152561, -86.831473)',
       '5610A GRANNY WHITE PIKE\nBRENTWOOD, TN 37027\n(36.046438, -86.815953)'],
      dtype=object)

In [35]:
# To pull out lat/lng from:
# '748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944, -86.879062)'
# Regular Expression - pattern matching
# RegEx link:  https://regex101.com/r/cAI6sh/1

pattern = re.compile(r'.*\((\d*\S\d*)\S\s(\S\d*\S\d*)\)', flags = re.MULTILINE)

def extract_lat_lon(map_loc):
    try:
        lat_lon_match = pattern.search(map_loc)
        lat = float(lat_lon_match.group(1))
        lon = float(lat_lon_match.group(2))
        return(lat, lon)
    except:
        return(np.NaN, np.NaN)

In [36]:
lat_lon = [extract_lat_lon(map_loc) for map_loc in df_bldg_apps.mapped_location]  #list comprehension

In [37]:
df_bldg_apps['lat'] = [lat for lat, lon in lat_lon]

In [38]:
df_bldg_apps['lon'] = [lon for lat, lon in lat_lon]

In [39]:
df_bldg_apps.head(2)

permit_number           permit_type_descr         permit_subtype_descr  \
0   T2020016213  Building Residential - New      Single Family Residence   
1   T2019073204      Building Moving Permit  Moving Permit - Residential   

        parcel date_entered  construction_cost            address  \
0  10216006100   2020-03-11                NaN     748  DARDEN PL   
1  04600002700   2019-12-02             2500.0  4836  BULL RUN RD   

           city state    zip                         subdivision_lot  \
0     NASHVILLE    TN  37205         LOT 168 SEC 9 PT 2 HILLWOOD EST   
1  ASHLAND CITY    TN  37015  N OF BULL RUN RD W OF OLD HICKORY BLVD   

                        contact permit_type permit_subtype  \
0  Kingdom Builders of Tennesse        CARN      CAA01R301   
1            CLAYTON HOMES #054        CAMV      CAZ09A001   

   ivr_tracking_number                                            purpose  \
0              3781725  New Single family dwelling.  REJECTED: APPLICA...   
1              3736813  Move existing mobile home from property out of...   

   council_district                                    mapped_location  \
0              23.0  748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944...   
1               1.0  4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36....   

         lat        lon  
0  36.125944 -86.879062  
1  36.242681 -86.929594

In [40]:
# Find out how many of the mapped locations had only the address, not the lat/lon
# Nulls in lat/lon 297 out of 3,106 = 9.5%
# To many to leave 'as is' Will try to add lat/lon fromUS Census geocoding (free) service:
# https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form

print(df_bldg_apps.shape)
print(df_bldg_apps.isnull().sum())

(3106, 20)
permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
construction_cost       1455
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            1
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                   22
council_district           7
mapped_location            0
lat                      297
lon                      297
dtype: int64


In [41]:
# Building Permits Issued

df_bldg_issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33909 entries, 0 to 33908
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   permit_number         33909 non-null  object        
 1   permit_type_descr     33909 non-null  object        
 2   permit_subtype_descr  33909 non-null  object        
 3   parcel                33909 non-null  object        
 4   date_entered          33909 non-null  datetime64[ns]
 5   date_issued           33909 non-null  datetime64[ns]
 6   construction_cost     33899 non-null  float64       
 7   address               33909 non-null  object        
 8   city                  33909 non-null  object        
 9   state                 33909 non-null  object        
 10  zip                   33909 non-null  int64         
 11  subdivision_lot       33909 non-null  object        
 12  contact               33908 non-null  object        
 13  permit_type     

In [42]:
# Review mapped_location in bldg_issued df before applying regex/function

df_bldg_issued.mapped_location.unique()

array(['1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218',
       '210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\n(36.042219, -86.764816)',
       '812 BRIAR CIR\nMADISON, TN 37115', ...,
       '131 EDENWOLD RD\nMADISON, TN 37115\n(36.287001, -86.703591)',
       '110 2ND AVE N\nNASHVILLE, TN 37201\n(36.162296, -86.77544)',
       '1382 RURAL HILL RD 320\nANTIOCH, TN 37013\n(36.056805, -86.649469)'],
      dtype=object)

In [43]:
# Applying function written for building applications to this building permits issued df

lat_lon = [extract_lat_lon(map_loc) for map_loc in df_bldg_issued.mapped_location]  

In [44]:
df_bldg_issued['lat'] = [lat for lat, lon in lat_lon]

In [45]:
df_bldg_issued['lon'] = [lon for lat, lon in lat_lon]

In [46]:
df_bldg_issued.head(2)

permit_number             permit_type_descr     permit_subtype_descr  \
0    2019070460    Building Residential - New  Single Family Residence   
1    2020016259  Building Residential - Rehab  Single Family Residence   

           parcel date_entered date_issued  construction_cost  \
0  058100C04900CO   2019-11-18  2019-12-09           270585.0   
1  160150A07000CO   2020-03-12  2020-03-12            12000.0   

                     address       city state  ...                contact  \
0     1037  LAWSONS RIDGE DR  NASHVILLE    TN  ...  CELEBRATION HOMES LLC   
1  210  HEARTHSTONE MANOR LN  BRENTWOOD    TN  ...  ACCESS & MOBILITY INC   

  permit_type permit_subtype ivr_tracking_number  \
0        CARN      CAA01R301             3733056   
1        CARR      CAA01R301             3781961   

                                             purpose  council_district  \
0  To construct a single family residence of 2402...               1.0   
1  to install a new elevator/platform lift from g...               4.0   

  census_tract                                    mapped_location        lat  \
0   37010105.0         1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218        NaN   
1   37018803.0  210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...  36.042219   

         lon  
0        NaN  
1 -86.764816  

[2 rows x 22 columns]

## Looked at addresses that have no lat/lon from Bldg Permit Application & Issued dfs
- Discovered there was a meaningful amount of relevant addresses that are missing lat/lon
- Will submit these to census tool to get lat/lon
- Chose to do this extra step because a meaninful number of addresses were missing lat/lon:
    - Bldg Permit Applications missing 297 out of 3,106 = 9.5% and 249 are new residential
    - Bldg Permits Issued missing 3,928 out of 33,909 = 11.5%

In [47]:
# Find out how many of the mapped locations had only the address, not the lat/lon: Bldg Permit Applications
# Number of rows missing lat & lon:  297

print(df_bldg_apps.shape)
print(df_bldg_apps.isnull().sum())

(3106, 20)
permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
construction_cost       1455
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            1
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                   22
council_district           7
mapped_location            0
lat                      297
lon                      297
dtype: int64


In [48]:
# Find out how many of the mapped locations had only the address, not the lat/lon: Bldg Permits Issued
# Number of rows missing lat & lon:  3928

print(df_bldg_issued.shape)
print(df_bldg_issued.isnull().sum())

(33909, 22)
permit_number              0
permit_type_descr          0
permit_subtype_descr       0
parcel                     0
date_entered               0
date_issued                0
construction_cost         10
address                    0
city                       0
state                      0
zip                        0
subdivision_lot            0
contact                    1
permit_type                0
permit_subtype             0
ivr_tracking_number        0
purpose                  467
council_district          46
census_tract              43
mapped_location            0
lat                     3928
lon                     3928
dtype: int64


In [49]:
# BLDG PERMIT APPLICATIONS
# First: Checked to see which items had null in lat/lon. Do I need these?
#        YES - there are a lot for new residential permits 
# Second: Created new df to submit to census geocoder website.

df_bldg_apps_null_latlon = df_bldg_apps.loc[df_bldg_apps['lat'].isnull()].reset_index(drop = True) 
df_bldg_apps_null_latlon.head(2)

permit_number           permit_type_descr     permit_subtype_descr  \
0   T2020034764  Building Residential - New  Single Family Residence   
1   T2020032881  Building Residential - New  Single Family Residence   

           parcel date_entered  construction_cost             address  \
0  165140A20100CO   2020-06-04                NaN    4929  CHUTNEY DR   
1  173100D02400CO   2020-05-27           260000.0  329  BODDINGTON LN   

      city state    zip               subdivision_lot  \
0  ANTIOCH    TN  37013  LOT 201 DAVENPORT DOWNS PH 2   
1  ANTIOCH    TN  37013      LOT 24 DELVIN DOWNS PH 6   

                            contact permit_type permit_subtype  \
0  AMH DEVELOPMENT TENNESSEE GC LLC        CARN      CAA01R301   
1                 CAPITOL HOMES INC        CARN      CAA01R301   

   ivr_tracking_number                                            purpose  \
0              3830089  to construct a single family residence with 22...   
1              3824736  New two story residential home in a approved P...   

   council_district                       mapped_location  lat  lon  
0              33.0    4929 CHUTNEY DR\nANTIOCH, TN 37013  NaN  NaN  
1              31.0  329 BODDINGTON LN\nANTIOCH, TN 37013  NaN  NaN

In [50]:
# BLDG PERMIT APPLICATIONS
# How many of these are meaningful in Bldg Permit Applications?   
# ANSWER: Most are important to know about for residential/commercial growth

df_bldg_apps_null_latlon.permit_type_descr.value_counts()

Building Residential - New                 249
Building Commercial - Tenant Finish Out     19
Building Use & Occupancy                    10
Building Sign Permit                         4
Building Commercial - Rehab                  4
Building Residential - Addition              3
Building Demolition Permit                   2
Building Commercial - New                    2
Building Residential Rehab Storm Damage      1
Building Blasting Permit                     1
Building Commercial Rehab Storm Damage       1
Building Residential - Roofing / Siding      1
Name: permit_type_descr, dtype: int64

In [51]:
# BLDG PERMITS ISSUED
# First: Checked to see which items had null in lat/lon. Do I need these? 
#        YES - there are a lot for new residential permits 
# Second: Created new df to submit to census geocoder website.

df_bldg_issued_null_latlon = df_bldg_issued.loc[df_bldg_issued['lat'].isnull()].reset_index(drop = True) 
df_bldg_issued_null_latlon.head(2)

permit_number           permit_type_descr     permit_subtype_descr  \
0    2019070460  Building Residential - New  Single Family Residence   
1    2019011084    Building Use & Occupancy    Multifamily, Townhome   

           parcel date_entered date_issued  construction_cost  \
0  058100C04900CO   2019-11-18  2019-12-09           270585.0   
1  051100J90000CO   2019-02-25  2019-07-22                0.0   

                  address       city state  ...                contact  \
0  1037  LAWSONS RIDGE DR  NASHVILLE    TN  ...  CELEBRATION HOMES LLC   
1          812  BRIAR CIR    MADISON    TN  ...   COLE INVESTMENTS LLC   

  permit_type permit_subtype ivr_tracking_number  \
0        CARN      CAA01R301             3733056   
1        CAUO      CAA03R301             3611315   

                                             purpose  council_district  \
0  To construct a single family residence of 2402...               1.0   
1  MASTER PERMIT ONLY� � NO CONSTRUCTION� �MULTI-...               8.0   

  census_tract                             mapped_location  lat lon  
0   37010105.0  1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218  NaN NaN  
1   37010802.0            812 BRIAR CIR\nMADISON, TN 37115  NaN NaN  

[2 rows x 22 columns]

In [52]:
# How many of these are meaningful in Bldg Permits Issued?   
# ANSWER: Most are important to know about for residential/commercial growth

df_bldg_issued_null_latlon.permit_type_descr.value_counts()

Building Residential - New                  3294
Building Commercial - Tenant Finish Out      172
Building Commercial - New                    100
Building Sign Permit                          74
Building Use & Occupancy                      57
Building Tree Removal Permit                  45
Building Commercial - Rehab                   43
Building Residential - Addition               25
Building Commercial - Foundation              19
Building Residential - Tenant Finish Out      18
Building Residential - Rehab                  14
Building Commercial - Shell                   14
Building Demolition Permit                    13
Building Residential - Amend Permit           12
Building Commercial - Addition                11
Building Commercial - Roofing / Siding         6
Building Blasting Permit                       6
Building Residential - Shell                   2
Building Residential - Change Contractor       1
Building Moving Permit                         1
Building Residential

### Creating new dfs for Bldg Permit Applications & Bldg Permits Issued with ONLY addresses that have no lat/lon, to submit to US Census Geocoder tool
- Will export to CSV; submit to census geocoder tool to get lat & lon, then add addresses back to original file (matching on addresss/city/st/zip columns)

In [53]:
df_bldg_apps_null_latlon.columns

Index(['permit_number', 'permit_type_descr', 'permit_subtype_descr', 'parcel',
       'date_entered', 'construction_cost', 'address', 'city', 'state', 'zip',
       'subdivision_lot', 'contact', 'permit_type', 'permit_subtype',
       'ivr_tracking_number', 'purpose', 'council_district', 'mapped_location',
       'lat', 'lon'],
      dtype='object')

In [54]:
# Dropping all but the address columns, in preparation for uploading to geocoder website

df_bldg_apps_null_latlon = df_bldg_apps_null_latlon[['address', 'city', 'state', 'zip']]
df_bldg_apps_null_latlon.head(2)

address     city state    zip
0    4929  CHUTNEY DR  ANTIOCH    TN  37013
1  329  BODDINGTON LN  ANTIOCH    TN  37013

In [55]:
df_bldg_issued_null_latlon = df_bldg_issued_null_latlon[['address', 'city', 'state', 'zip']]
df_bldg_issued_null_latlon.head(2)

address       city state    zip
0  1037  LAWSONS RIDGE DR  NASHVILLE    TN  37218
1          812  BRIAR CIR    MADISON    TN  37115

In [56]:
df_bldg_apps_null_latlon.to_csv('../data/interim/bldg_apps_addresses.csv')

In [57]:
df_bldg_issued_null_latlon.to_csv('../data/interim/bldg_issued_addresses.csv')

### Aborted the attempt to get lat/lon on 6/12/2020. Census Geocoder tool only found 1% of addresses.
- The census geocoder tool only found 1% of addresses in each file as exact match, and additional 1% in one of the files as non-exact match - but the remaining 98-99% weren't found.
- Aborting this process becasue it isn't worth the time to fuss with clean and import just 1%. It would be cumbersome because:
    - Addresses were submitted as 4 columns for address; but returned with address, city, state and zip all concatenated in a single cell.
    - Returns [lat, lon] in single column, in addition to quite a bit of other information in other columns I don't need.
- Returned files are in ../data/interim folder for future reference
- US Census website, for reference: https://geocoding.geo.census.gov/geocoder/geographies/addressbatch?form

### Creating new columns for rolling week, month, quarter, year in Bldg Permit Applications & Issued, and Planning Dept Permits 

### ABORTED rolling date idea on 6/12/2020:
- Discovered that it won't work due to needing to first group by category (Residential, Commercial, Other).
- It'll be better to do groupby, counts, etc. in the 2_eda notebook as needed, and do additional slicing in Tableau later on.

## Exploring and Cleaning: Planning Dept Applications / Issued

In [58]:
# Checking nulls, building permit applications

df_planning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_submitted              521 non-null    datetime64[ns]
 1   application_type_descr      521 non-null    object        
 2   mpc_case_number             520 non-null    object        
 3   ordinance_number            157 non-null    object        
 4   status                      521 non-null    object        
 5   mpc_meeting_date            521 non-null    datetime64[ns]
 6   mpc_action                  235 non-null    object        
 7   project_name                417 non-null    object        
 8   location                    498 non-null    object        
 9   reviewer                    521 non-null    object        
 10  reviewer_email              465 non-null    object        
 11  case_descr                  503 non-null    object        

In [59]:
# Taking a look at the types of info in Planning Dept data

print(df_planning.application_type_descr.nunique())
print(df_planning.application_type_descr.value_counts())

31
Rezoning                                    88
Subdivision (Final Plat)                    86
Mandatory Referral Easement                 65
Specific Plan (Final Site Plan)             50
Specific Plan (New)                         40
Mandatory Referral Encroachment             24
Community Plan Amendment                    17
Mandatory Referral Agreement                15
Planned Unit Development (Final Site Pl)    15
Text Amendment                              14
Downtown Code (Final Site Plan)             13
Subdivision (Concept Plan)                   9
Planned Unit Development (Cancel)            8
Downtown Code (Modify)                       8
Mandatory Referral Property                  8
Specific Plan (Amend)                        8
Subdivision (Amendment)                      8
Planned Unit Development (Amend)             8
Mandatory Referral  R.O.W. Abandonment       7
Urban Design Overlay (Final)                 6
Historic Landmark (New)                      5
Urban Desi

In [60]:
# Taking a look at the types of info in Planning Dept data

print(df_planning.status.nunique())
print(df_planning.status.value_counts())

5
PENDING        220
CNCLACTIVE     207
NEW             71
MPCCOMPLETE     22
UNKNOWN          1
Name: status, dtype: int64


## Exploring and Cleaning: Neighborhood Assoc Boundaries (GIS)
- The only change needed was to fix the crs code from epsg:4326 go EPSG:4326

In [61]:
df_na_bound.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 288 entries, 0 to 287
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      288 non-null    object  
 1   geometry  288 non-null    geometry
dtypes: geometry(1), object(1)
memory usage: 4.6+ KB


In [62]:
# To confirm whether any are null.  No nulls! Good :-)

df_na_bound.isnull().sum()

name        0
geometry    0
dtype: int64

In [63]:
# Confirming crs type

print(df_na_bound.crs)

epsg:4326


In [64]:
# Converting to uppercase EPSG

df_na_bound.crs = "EPSG:4326"
print(df_na_bound.crs)

EPSG:4326


## Review permit types, permit subtypes, etc. in Bldg Permit Applications, Bldg Permits Issued, and Planning dfs.
- Are there any types, subtypes, or columns not needed for EDA?

In [65]:
# Bldg Permit Applications
# Which columns not needed?  
# KEEPING ALL. Might want them for popups on visualization

df_bldg_apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3106 entries, 0 to 3105
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   permit_number         3106 non-null   object        
 1   permit_type_descr     3106 non-null   object        
 2   permit_subtype_descr  3106 non-null   object        
 3   parcel                3106 non-null   object        
 4   date_entered          3106 non-null   datetime64[ns]
 5   construction_cost     1651 non-null   float64       
 6   address               3106 non-null   object        
 7   city                  3106 non-null   object        
 8   state                 3106 non-null   object        
 9   zip                   3106 non-null   int64         
 10  subdivision_lot       3105 non-null   object        
 11  contact               3105 non-null   object        
 12  permit_type           3106 non-null   object        
 13  permit_subtype    

In [66]:
# Bldg Permit Applications
# Which PERMIT TYPES are needed to answer data questions?
# DECISION: KEEPING ALL
# REASON: Even though some types only have one count,
#         several with one value may be in a single neighborhood assoc boundary
#         May group this once I start exploring, but will leave all details in place for now.

df_bldg_apps.permit_subtype_descr.value_counts()

Single Family Residence                     1420
Demolition Permit - Residential              173
Accessory Structure, Garage                  167
Sign - Ground /  Wall Signs                  136
Tents, Stages                                 99
                                            ... 
Automobile Convenience, Markets                1
Multi-Media Production                         1
Community Education, Stadiums                  1
Hospice, Institutional I-2                     1
Telephone Services - Cell Towers & Co-Lo       1
Name: permit_subtype_descr, Length: 94, dtype: int64

In [67]:
# Bldg Permit Applications
# Which PERMIT SUB-TYPES are needed to answer data questions?
# DECISION: KEEPING ALL
# REASON: Even though some types only have one count,
#         several with one value may be in a single neighborhood assoc boundary
#         May group this once I start exploring, but will leave all details in place for now.

# To get full results that aren't truncated, set to 4000:
# pd.options.display.max_rows = 50
# print(pd.options.display.max_rows)

list_bldg_apps_subtype_counts = df_bldg_apps.permit_subtype_descr.value_counts()
print(list_bldg_apps_subtype_counts)

Single Family Residence                     1420
Demolition Permit - Residential              173
Accessory Structure, Garage                  167
Sign - Ground /  Wall Signs                  136
Tents, Stages                                 99
                                            ... 
Automobile Convenience, Markets                1
Multi-Media Production                         1
Community Education, Stadiums                  1
Hospice, Institutional I-2                     1
Telephone Services - Cell Towers & Co-Lo       1
Name: permit_subtype_descr, Length: 94, dtype: int64


In [68]:
# Bldg Permits Issued
# Which columns not needed?  
# DECISION: KEEPING ALL
# REASON: Might want this information for popups on visualization

df_bldg_issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33909 entries, 0 to 33908
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   permit_number         33909 non-null  object        
 1   permit_type_descr     33909 non-null  object        
 2   permit_subtype_descr  33909 non-null  object        
 3   parcel                33909 non-null  object        
 4   date_entered          33909 non-null  datetime64[ns]
 5   date_issued           33909 non-null  datetime64[ns]
 6   construction_cost     33899 non-null  float64       
 7   address               33909 non-null  object        
 8   city                  33909 non-null  object        
 9   state                 33909 non-null  object        
 10  zip                   33909 non-null  int64         
 11  subdivision_lot       33909 non-null  object        
 12  contact               33908 non-null  object        
 13  permit_type     

In [69]:
# Bldg Permits Issued
# Which PERMIT TYPES are needed to answer data questions?
# DECISION: KEEPING ALL
# REASON: Even though some types only have small counts,
#         several may be in a single neighborhood assoc boundary
#         May group this once I start exploring, but will leave all details in place for now.

df_bldg_issued.permit_type_descr.value_counts()

Building Residential - New                  12780
Building Commercial - Rehab                  3384
Building Residential - Rehab                 3273
Building Residential - Addition              3200
Building Demolition Permit                   2796
Building Sign Permit                         2436
Building Use & Occupancy                     1524
Building Commercial - Tenant Finish Out      1113
Building Commercial - New                    1047
Building Tree Removal Permit                  495
Building Commercial - Addition                338
Building Residential - Roofing / Siding       240
Building Commercial - Roofing / Siding        239
Building Commercial - Shell                   173
Building Residential Rehab Storm Damage       119
Building Residential - Tenant Finish Out      117
Building Commercial - Foundation              114
Building Blasting Permit                      107
Building Residential - Change Contractor       91
Building Residential - Fire Damage             90


In [70]:
# Bldg Permits Issued
# Which PERMIT SUB-TYPES are needed to answer data questions?
# DECISION: KEEPING ALL
# REASON: Even though some types only have small counts,
#         several may be in a single neighborhood assoc boundary
#         May group this once I start exploring, but will leave all details in place for now.

list_bldg_issued_subtype_counts = df_bldg_issued.permit_subtype_descr.value_counts()
print(list_bldg_issued_subtype_counts)

Single Family Residence                    15399
Demolition Permit - Residential             2339
Sign - Ground /  Wall Signs                 2313
Multifamily, Townhome                       1346
General Office, Professional Services       1308
                                           ...  
Google Fiber Huts                              1
Recreation Center, Exhibition Halls            1
Building Contractor Supply, Storage S-2        1
Historical Home Events, Other R-3              1
Comm. Amusement (Out), Bleachers               1
Name: permit_subtype_descr, Length: 178, dtype: int64


In [71]:
# Planning / Zoning Dept. Applications and Permits Issued
# This data shows ALL PENDING, and only the LAST TWO MONTHS of ISSUED.
# ACTIONS TAKEN BELOW: 
#    1. Converted date fields to datetime
#    2. Dropped the two columns that were all null values

df_planning.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521 entries, 0 to 520
Data columns (total 27 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   date_submitted              521 non-null    datetime64[ns]
 1   application_type_descr      521 non-null    object        
 2   mpc_case_number             520 non-null    object        
 3   ordinance_number            157 non-null    object        
 4   status                      521 non-null    object        
 5   mpc_meeting_date            521 non-null    datetime64[ns]
 6   mpc_action                  235 non-null    object        
 7   project_name                417 non-null    object        
 8   location                    498 non-null    object        
 9   reviewer                    521 non-null    object        
 10  reviewer_email              465 non-null    object        
 11  case_descr                  503 non-null    object        

In [72]:
# Convert date_submitted to datetime 

df_planning.date_submitted = pd.to_datetime(df_planning.date_submitted)
df_planning.date_submitted.head(2)

0   2019-04-01
1   2019-11-27
Name: date_submitted, dtype: datetime64[ns]

In [73]:
# Convert mpc_meeting_date to datetime 

df_planning.mpc_meeting_date = pd.to_datetime(df_planning.mpc_meeting_date)
df_planning.mpc_meeting_date.head(2)

0   2020-06-11
1   2020-01-16
Name: mpc_meeting_date, dtype: datetime64[ns]

In [74]:
# To count nulls

df_planning.isnull().sum()

date_submitted                  0
application_type_descr          0
mpc_case_number                 1
ordinance_number              364
status                          0
mpc_meeting_date                0
mpc_action                    286
project_name                  104
location                       23
reviewer                        0
reviewer_email                 56
case_descr                     18
applicant                      20
applicant_representative       24
applicant_email                25
applicant_phone                23
applicant_address_1            24
applicant_address_2           382
applicant_city                 24
applicant_state                24
applicant_zip                  24
council_3rd_reading_date      521
council_3rd_reading_action    521
council_district               32
latitude                       32
longitude                      32
mapped_location                32
dtype: int64

In [75]:
# Dropping the two columns for '...3rd_reading_...' that have all null values. Keeping same df name.

df_planning = df_planning.drop(columns = ['council_3rd_reading_date'
                                          , 'council_3rd_reading_action'
                                         ])
df_planning.isnull().sum()

date_submitted                0
application_type_descr        0
mpc_case_number               1
ordinance_number            364
status                        0
mpc_meeting_date              0
mpc_action                  286
project_name                104
location                     23
reviewer                      0
reviewer_email               56
case_descr                   18
applicant                    20
applicant_representative     24
applicant_email              25
applicant_phone              23
applicant_address_1          24
applicant_address_2         382
applicant_city               24
applicant_state              24
applicant_zip                24
council_district             32
latitude                     32
longitude                    32
mapped_location              32
dtype: int64

In [76]:
# Confirming that the columns were dropped.

df_planning.columns

Index(['date_submitted', 'application_type_descr', 'mpc_case_number',
       'ordinance_number', 'status', 'mpc_meeting_date', 'mpc_action',
       'project_name', 'location', 'reviewer', 'reviewer_email', 'case_descr',
       'applicant', 'applicant_representative', 'applicant_email',
       'applicant_phone', 'applicant_address_1', 'applicant_address_2',
       'applicant_city', 'applicant_state', 'applicant_zip',
       'council_district', 'latitude', 'longitude', 'mapped_location'],
      dtype='object')

In [77]:
# Looking at value counts for mpc (Municipal Planning Committee) actions
# Choosing, again, to leave them all in place, for now.

df_planning.mpc_action.value_counts()

Recommend Approval                         99
Approved by MPC                            62
Approve with Conditions                    59
Deferred Indefinitely by App at MPC         4
Deferred Indefinitely by App before MPC     2
Approved by Executive Director              2
Withdrawn                                   2
Neutral / No Position                       1
Disapprove                                  1
Disapprove with Conditions                  1
Deferred by Applic before MPC               1
Deferred by MPC                             1
Name: mpc_action, dtype: int64

In [78]:
# Status has no nulls: This includes every entry
# mpc_action, above, is apparently only filled in when an action is taken.

df_planning.status.value_counts()

PENDING        220
CNCLACTIVE     207
NEW             71
MPCCOMPLETE     22
UNKNOWN          1
Name: status, dtype: int64

## Making new category column, for Residential, Commercial, Other

In [79]:
# Building Permit Applications df
# Stackoverflow resource: https://stackoverflow.com/questions/36653419/str-contains-to-create-new-column-in-pandas-dataframe

# Set a default value for new category column
df_bldg_apps['category'] = 'Other'

# Assign Commercial, Residential based on permit_type AND permit_subtype, to capture the most possible of each one
# Doing BOTH type and sub_type increased Commercial category the most
df_bldg_apps.loc[df_bldg_apps['permit_type_descr'].str.contains('Commercial'), 'category'] = 'Commercial'
df_bldg_apps.loc[df_bldg_apps['permit_subtype_descr'].str.contains('Commercial'), 'category'] = 'Commercial'

df_bldg_apps.loc[df_bldg_apps['permit_type_descr'].str.contains('Residential'), 'category'] = 'Residential'
df_bldg_apps.loc[df_bldg_apps['permit_subtype_descr'].str.contains('Residential'), 'category'] = 'Residential'

df_bldg_apps['category'].value_counts()

Residential    2204
Commercial      458
Other           444
Name: category, dtype: int64

In [80]:
# Building Permits Issued df
# Stackoverflow resource: https://stackoverflow.com/questions/36653419/str-contains-to-create-new-column-in-pandas-dataframe

# Set a default value for new category column
df_bldg_issued['category'] = 'Other'

# Assign Commercial, Residential based on permit_type AND permit_subtype, to capture the most possible of each one
# Doing BOTH type and sub_type increased Commercial category the most
df_bldg_issued.loc[df_bldg_issued['permit_type_descr'].str.contains('Commercial'), 'category'] = 'Commercial'
df_bldg_issued.loc[df_bldg_issued['permit_subtype_descr'].str.contains('Commercial'), 'category'] = 'Commercial'

df_bldg_issued.loc[df_bldg_issued['permit_type_descr'].str.contains('Residential'), 'category'] = 'Residential'
df_bldg_issued.loc[df_bldg_issued['permit_subtype_descr'].str.contains('Residential'), 'category'] = 'Residential'

df_bldg_issued['category'].value_counts()

Residential    22893
Commercial      7029
Other           3987
Name: category, dtype: int64

In [81]:
# Confirm new column added (at end)

df_bldg_issued.head(2)

permit_number             permit_type_descr     permit_subtype_descr  \
0    2019070460    Building Residential - New  Single Family Residence   
1    2020016259  Building Residential - Rehab  Single Family Residence   

           parcel date_entered date_issued  construction_cost  \
0  058100C04900CO   2019-11-18  2019-12-09           270585.0   
1  160150A07000CO   2020-03-12  2020-03-12            12000.0   

                     address       city state  ...  permit_type  \
0     1037  LAWSONS RIDGE DR  NASHVILLE    TN  ...         CARN   
1  210  HEARTHSTONE MANOR LN  BRENTWOOD    TN  ...         CARR   

  permit_subtype ivr_tracking_number  \
0      CAA01R301             3733056   
1      CAA01R301             3781961   

                                             purpose council_district  \
0  To construct a single family residence of 2402...              1.0   
1  to install a new elevator/platform lift from g...              4.0   

   census_tract                                    mapped_location        lat  \
0    37010105.0         1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218        NaN   
1    37018803.0  210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\...  36.042219   

         lon     category  
0        NaN  Residential  
1 -86.764816  Residential  

[2 rows x 23 columns]

In [82]:
df_planning.head(2)

date_submitted           application_type_descr mpc_case_number  \
0     2019-04-01         Subdivision (Final Plat)   2019S-086-001   
1     2019-11-27  Specific Plan (Final Site Plan)  2016SP-076-008   

  ordinance_number   status mpc_meeting_date mpc_action  \
0              NaN  PENDING       2020-06-11        NaN   
1              NaN  PENDING       2020-01-16        NaN   

                                        project_name              location  \
0  FINAL PLAT RESUBDIVISION OF LOT 3 AND 4 ON THE...  227 MARCIA AVE 37209   
1                                 RED OAKS TOWNHOMES        0 DEW ST 37206   

          reviewer  ... applicant_phone  applicant_address_1  \
0  Joren Dunnavant  ...    615-490-3236    1711 Hayes Street   
1    Abbie Rickoff  ...    615-351-3634  214 Oceanside Drive   

  applicant_address_2 applicant_city applicant_state applicant_zip  \
0                 NaN      Nashville              TN         37203   
1                 NaN      Nashville              TN         37204   

            council_district   latitude  longitude  \
0  20 (Mary Carolyn Roberts)  36.143923 -86.868254   
1         06 (Brett Withers)  36.165962 -86.753490   

                            mapped_location  
0  (36.143922831000054, -86.86825400699996)  
1  (36.165961579000054, -86.75348957099999)  

[2 rows x 25 columns]

In [83]:
# Planning Dept df
# This df doesn't reference commercial vs residential, so 'category' column isn't applicable.

df_planning.application_type_descr.value_counts()

Rezoning                                    88
Subdivision (Final Plat)                    86
Mandatory Referral Easement                 65
Specific Plan (Final Site Plan)             50
Specific Plan (New)                         40
Mandatory Referral Encroachment             24
Community Plan Amendment                    17
Mandatory Referral Agreement                15
Planned Unit Development (Final Site Pl)    15
Text Amendment                              14
Downtown Code (Final Site Plan)             13
Subdivision (Concept Plan)                   9
Planned Unit Development (Cancel)            8
Downtown Code (Modify)                       8
Mandatory Referral Property                  8
Specific Plan (Amend)                        8
Subdivision (Amendment)                      8
Planned Unit Development (Amend)             8
Mandatory Referral  R.O.W. Abandonment       7
Urban Design Overlay (Final)                 6
Historic Landmark (New)                      5
Urban Design 

In [84]:
df_planning.status.value_counts()

PENDING        220
CNCLACTIVE     207
NEW             71
MPCCOMPLETE     22
UNKNOWN          1
Name: status, dtype: int64

### Sorting dfs by dates, descending.

In [85]:
# Bldg Permit Applications: Sorting by date_entered, descending.

df_bldg_apps = df_bldg_apps.sort_values(by = 'date_entered', ascending = False)
df_bldg_apps.head(2)

permit_number             permit_type_descr     permit_subtype_descr  \
563    T2020034761    Building Residential - New  Single Family Residence   
1033   T2020034642  Building Residential - Rehab  Single Family Residence   

           parcel date_entered  construction_cost          address       city  \
563   08211018000   2020-06-04                NaN     518 N 2ND ST  NASHVILLE   
1033  09115008600   2020-06-04            80000.0  221  53RD AVE N  NASHVILLE   

     state    zip  ...                  contact permit_type permit_subtype  \
563     TN  37207  ...  EASTEND CONTRACTING LLC        CARN      CAA01R301   
1033    TN  37209  ...       THE KINGSTON GROUP        CARR      CAA01R301   

     ivr_tracking_number                                            purpose  \
563              3830078  to construct 2034SF single family residence. 5...   
1033             3829667  Finish out a bonus room (that is already frame...   

     council_district                                    mapped_location  \
563               5.0  518 N 2ND ST\nNASHVILLE, TN 37207\n(36.18049, ...   
1033             24.0  221 53RD AVE N\nNASHVILLE, TN 37209\n(36.14734...   

            lat        lon     category  
563   36.180490 -86.771778  Residential  
1033  36.147342 -86.850887  Residential  

[2 rows x 21 columns]

In [86]:
# Double-checking min/max in this df

print(df_bldg_apps.date_entered.min())
print(df_bldg_apps.date_entered.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


In [87]:
# Bldg Permits Issued: Sorting by date_issued, descending.

df_bldg_issued = df_bldg_issued.sort_values(by = 'date_issued', ascending = False)
df_bldg_issued.head(2)

permit_number                       permit_type_descr  \
12549    2020032857  Building Commercial Rehab Storm Damage   
12552    2020033336            Building Residential - Rehab   

                         permit_subtype_descr       parcel date_entered  \
12549  Multifamily, Condominium 3&4 Unit Bldg  08214005500   2020-05-27   
12552                 Single Family Residence  07309021300   2020-05-29   

      date_issued  construction_cost             address       city state  \
12549  2020-06-04           800000.0        186 N 1ST ST  NASHVILLE    TN   
12552  2020-06-04            80000.0  1913  ROSEBANK AVE  NASHVILLE    TN   

       ...  permit_type permit_subtype ivr_tracking_number  \
12549  ...         CACL      CAA03R298             3824603   
12552  ...         CARR      CAA01R301             3826163   

                                                 purpose council_district  \
12549  ,there is some remodeling to be performed in t...              5.0   
12552  General rehabilitation of home within existing...              7.0   

       census_tract                                    mapped_location  \
12549    37019300.0  186 N 1ST ST\nNASHVILLE, TN 37213\n(36.173878,...   
12552    37011500.0  1913 ROSEBANK AVE\nNASHVILLE, TN 37216\n(36.19...   

             lat        lon     category  
12549  36.173878 -86.774064   Commercial  
12552  36.198145 -86.704423  Residential  

[2 rows x 23 columns]

In [88]:
# Double-checking min/max in this df

print(df_bldg_issued.date_issued.min())
print(df_bldg_issued.date_issued.max())

2017-06-01 00:00:00
2020-06-04 00:00:00


In [89]:
# Planning Dept. Applications & Issued: Sorting by mpc_meeting_date, descending.
# Chose mpc_meeting_date because that 

df_planning = df_planning.sort_values(by = 'mpc_meeting_date', ascending = False)
df_planning.head(2)

date_submitted                  application_type_descr  mpc_case_number  \
306     2018-01-17                     Specific Plan (New)   2018SP-027-001   
377     2020-05-26  Mandatory Referral  R.O.W. Abandonment  2020M-008AB-001   

    ordinance_number status mpc_meeting_date mpc_action  \
306       BL2020-221    NEW       2020-07-23        NaN   
377              NaN    NEW       2020-07-23        NaN   

                                          project_name  \
306                             12558 OLD HICKORY BLVD   
377  UNNUMBERED ALLEY (OFF CENTER STREET) RIGHT-OF-...   

                         location           reviewer  ... applicant_phone  \
306  12558 OLD HICKORY BLVD 37013  Latisha Birkeland  ...    615-297-5166   
377             0 CENTER ST 37138    Sharon O'Conner  ...    615-862-8781   

        applicant_address_1 applicant_address_2 applicant_city  \
306       516 HEATHER PLACE                 NaN      NASHVILLE   
377  720 SOUTH FIFTH STREET                 NaN      NASHVILLE   

    applicant_state applicant_zip     council_district   latitude  longitude  \
306              TN         37204  33 (Antoinette Lee)  36.035350 -86.604835   
377              TN         37206     11 (Larry Hagar)  36.225994 -86.629702   

                              mapped_location  
306  (36.035350452000046, -86.60483534099995)  
377   (36.22599424900005, -86.62970214499995)  

[2 rows x 25 columns]

In [90]:
# Double-checking min/max dates in this df

print("Min date_submitted is: ",df_planning.date_submitted.min())
print("Max date_submitted is: ",df_planning.date_submitted.max())
print("Min mpc_meeting_date is: ",df_planning.mpc_meeting_date.min())
print("Max mpc_meeting_date is: ",df_planning.mpc_meeting_date.max())    # Future date is correct

Min date_submitted is:  2017-02-28 00:00:00
Max date_submitted is:  2020-06-04 00:00:00
Min mpc_meeting_date is:  2017-04-13 00:00:00
Max mpc_meeting_date is:  2020-07-23 00:00:00


## LAST STEP: Save output files in .. / data / processed / filename_clean

In [95]:
# Save Building Permit Applications file to data\cleaned folder
df_bldg_apps.to_csv('../data/cleaned/bldg_permit_applications_clean.csv')

# Printing shape, will add to 2_eda notebook to validate it when it's read in
df_bldg_apps.shape

(3106, 21)

In [96]:
df_bldg_issued.to_csv('../data/cleaned/bldg_permits_issued_clean.csv')
df_bldg_issued.shape

(33909, 23)

In [97]:
df_planning.to_csv('../data/cleaned/planning_dept_clean.csv')
df_planning.shape

(521, 25)

In [98]:
df_na_bound.to_file('../data/cleaned/neighborhood_association_boundaries_clean.shp')
df_na_bound.shape

(288, 2)